In [1]:
import pandas as pd

data_path = '../data'
benchmark = 'zc_transbench101_macro'

In [2]:
from zc_combine.utils.naslib_utils import load_search_space, parse_scores

search_space = load_search_space(data_path, benchmark)
dfs = parse_scores(search_space)

In [3]:
dfs.keys()

dict_keys(['class_scene', 'class_object', 'autoencoder', 'normal', 'jigsaw', 'room_layout', 'segmentsemantic'])

In [4]:
dataset = 'class_scene'
data = dfs[dataset]

In [5]:
# channel change: [2, 4]; stride change: [3, 4]

In [6]:
data

,net,val_accs,fisher,flops,grad_norm,grasp,jacov,l2_norm,nwot,params,plain,snip,zen,epe_nas,synflow,random,rank
0,"(2, 3, 2, 3, 2, 3)",54.488457,1.224128,8412.831744,26.729975,5.725534,-32.010033,680.217773,494.165980,23.810143,0.003494,294.458923,92.824593,568.254794,106.095618,129,828.0
1,"(3, 1, 1, 2, 1, 2)",52.040207,0.347729,3847.127040,7.749443,-0.124374,-32.011106,396.607269,478.706460,3.699935,0.001887,60.061455,76.052307,557.312114,95.667023,2291,2172.5
2,"(4, 4, 1, 2, 1, 1)",53.025478,1.788998,8757.624832,19.600498,1.069871,-32.015042,673.558594,477.264658,32.314719,0.049673,238.195007,88.925102,329.478267,108.859891,2891,1665.0
3,"(4, 3, 3, 1, 3, 2)",50.746418,1.512442,906.483712,34.669132,10.072220,-32.010837,489.532349,448.223581,5.074271,0.081322,274.857574,79.667213,502.820006,97.333634,1141,2692.5
4,"(2, 1, 1, 3, 2, 3)",55.573250,0.547060,10486.702080,12.762392,0.243145,-32.010186,507.085022,511.857239,6.877535,0.036490,115.483612,83.242722,492.245915,101.301427,2008,298.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3251,"(4, 1, 1, 3, 4, 4)",51.671974,1.367546,2266.288128,32.988228,5.798134,-32.011667,534.842163,468.713476,12.859999,0.058923,304.852448,81.559113,564.391043,101.087128,706,2335.5
3252,"(1, 4, 1, 2, 4, 3)",55.374207,1.153950,6765.125632,22.905090,1.906332,-32.011826,595.817810,488.912078,21.514079,0.080528,237.685104,85.060875,674.621578,103.341069,1572,386.5
3253,"(2, 3, 4, 3, 4)",53.383759,1.733507,3697.004544,34.260513,12.492985,-32.015507,519.842346,481.858619,14.105695,-0.018997,328.126099,74.797157,638.571885,87.751223,1633,1447.0
3254,"(4, 3, 2, 1, 4)",52.562698,0.780785,2466.291712,16.966343,1.523422,-32.015158,497.444519,460.629998,14.039647,0.064737,168.335815,73.851662,511.464159,88.476610,3021,1915.0


In [7]:
def get_encoding(net):
    res = []
    net = net.strip('()').split(',')

    for idx in net:
        encoding = {}
        idx = int(idx)
        encoding['channel'] = idx % 2 == 0
        encoding['stride'] = idx > 2
        res.append(encoding)
    
    return res

In [12]:
enc = get_encoding(data.iloc[0]['net'])
enc

[{'channel': True, 'stride': False},
 {'channel': False, 'stride': True},
 {'channel': True, 'stride': False},
 {'channel': False, 'stride': True},
 {'channel': True, 'stride': False},
 {'channel': False, 'stride': True}]

In [13]:
def get_state_at_i(net):
    channel = 0
    stride = 0
    
    res = {}
    for i in range(6):
        if i >= len(net):
            channel = 0
            stride = 0
        else:
            channel += int(net[i]['channel'])
            stride += int(net[i]['stride'])
        
        res[f"ch_pos_{i}"] = channel
        res[f"s_pos_{i}"] = stride
    
    return res

In [14]:
get_state_at_i(enc)

{'ch_pos_0': 1,
 's_pos_0': 0,
 'ch_pos_1': 1,
 's_pos_1': 1,
 'ch_pos_2': 2,
 's_pos_2': 1,
 'ch_pos_3': 2,
 's_pos_3': 2,
 'ch_pos_4': 3,
 's_pos_4': 2,
 'ch_pos_5': 3,
 's_pos_5': 3}